# FTSE 4 HR BACKTEST PROJECT

I this notebook we will build features using multiple arguments which will assist when developing our interactive application.

In the earlier feature engineering notebook, our setpoints such as target, stop loss and draw down which, while utilising sensible values, is not representitive of the flexibility required in reality. 

We'll re-start using the base .csv file exported from Trading View 

In [14]:
#import libraries

import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")

In [15]:
import ftse_functions
from ftse_functions import outcome_generator, direction_calc, dataframe

In [19]:
#check our directory contents
!ls

FTSE4 Data Prep.ipynb      ftse_base4.csv
FTSE4.csv                  ftse_flexi_functions.ipynb
FTSE_ma_forecaster.ipynb   ftse_functions.ipynb
README.md                  ftse_functions.py
__pycache__                ftse_static.csv
ftse_4_features.ipynb      ftse_user_interface.ipynb


In [20]:
#Read in the data set
df = pd.read_csv("ftse_base4.csv", parse_dates=["time"])

In [21]:
#review the top of the dataframe
df.head()

,time,open,high,low,close
0,2020-04-23 11:00:00,5770.0,5822.0,5755.6,5798.1
1,2020-04-23 15:00:00,5797.9,5847.9,5762.1,5773.2
2,2020-04-23 19:00:00,5773.2,5796.8,5744.1,5769.5
3,2020-04-23 23:00:00,5756.8,5770.2,5731.6,5740.2
4,2020-04-24 03:00:00,5740.0,5755.0,5720.3,5744.3


## Feature Engineering - static features

Now we can re-use some of the initial code as some of the features are fixed including the direction and signal features. 

### Direction Feature

In [22]:
#Create the direction feature
def direction(row):
    """
    In this function we will look at the values in each row of the data set and if the close value is lower than
    the open value we'll assign "short" and if the close value is higher than the open value we'll assign "long".
    If the close value equals the open value we'll assign neutral.
    """
    
    #Assign the columns to variables. so the column at position 1 is the open price so we assign to open_. 
    open_ = row[1]
    close = row[4]
    
    #Write the conditional that will determine the direction to be returned. 
    if open_ < close:
        return  'long'
    elif open_ > close:
        return 'short'
    else:
        return 'neutral'

In [23]:
#Write a .apply statement to iterate thro' all rows in our dataset and return candle direction into our new feature
market_direction = df.apply(direction_calc, axis='columns').to_frame()

#join our main df to the newly created market_direction dataframe
df=df.join(market_direction)

#The new feature is automatically given a header name of 0. Let's change this to something more descriptive
df.rename(columns={0:'direction'}, inplace=True)

In [24]:
df.head()

,time,open,high,low,close,direction
0,2020-04-23 11:00:00,5770.0,5822.0,5755.6,5798.1,long
1,2020-04-23 15:00:00,5797.9,5847.9,5762.1,5773.2,short
2,2020-04-23 19:00:00,5773.2,5796.8,5744.1,5769.5,short
3,2020-04-23 23:00:00,5756.8,5770.2,5731.6,5740.2,short
4,2020-04-24 03:00:00,5740.0,5755.0,5720.3,5744.3,long


### Signal Feature

In [25]:
#Create the signal feature

#Set n variable that we can manipulate from within the function
x = 0 

def signal(row):
    """
    In this funtion we will look at pairs of rows. If the current row has the same directon as the previous row or 
    has neutral direction we'll assign "no trade" to the new feature. Else we'll assign "trade".
    """
    #We need access to the global variable so we can increment the value as we iterate through the rows
    global x
    
    #We have no access to earlier data so we must assign "unknown" to the first row
    if x == 0:
        x+=1
        return "unknown"
    
    #write the conditional to check if we have a valid trade signal
    if df.loc[x].direction == df.loc[x-1].direction:
        x += 1
        return "no_trade"
    elif df.loc[x].direction == "neutral":
        x += 1
        return "no_trade"
    else:
        x+=1
        return "trade"

In [26]:
#Pass our assigned signals to a new dataframe
trade_sig = df.apply(signal, axis='columns').to_frame()

#join the new signal df to our main df
df=df.join(trade_sig)

#Give the new feature column a descriptive header
df.rename(columns={0:'signal'}, inplace=True)

In [27]:
#Quick review of the two added features
df.head(2)

,time,open,high,low,close,direction,signal
0,2020-04-23 11:00:00,5770.0,5822.0,5755.6,5798.1,long,unknown
1,2020-04-23 15:00:00,5797.9,5847.9,5762.1,5773.2,short,trade


In [134]:
df.signal.value_counts()

no_trade    336
trade       326
unknown       1
Name: signal, dtype: int64

In [28]:
#df.to_csv('ftse_static.csv', index=False)

********************************************************

## Feature Engineering - Dynamic Features

Here we'll create the features that rely on user input to generate the returned values. Note that the dynamic functions will be copy and pasted into a file named ftse_functions.py so they can be imported by other programmes and to help keep the notebooks clear and concise. 

### Outcome Feature

Now for this feature we previously had to hard code a bespoke function for each target strategy, but by adding multiple arguments which we can subsequently call we have the flexibility within a single function.

In [ ]:
#Create the flexible outcome feature

#copy the exisiting df in to a new dataframe to make the function easily recyclable
df1 = df

#Set a variable that we can increment from within the function
x = 0 

def outcome_generator(row, target, drawdown):
   
    global x #let's make our variable global so we can change it's value from inside this function
    
    
    #We'll add a print report to show the progress we're making mid function execute
    if x%200 == 0: 
        print("processing row {}".format(x))
    
    
    #create variables for each of the rows we may need to iterate over
    high = df["high"]
    low = df["low"]
    close = df["close"]
    open_ = df["open"]
    signal = df["signal"]
    direction = df["direction"]
    
   
    
    long_short = "" #set an empty variable to take a long or short position 
    
#     if x == 0: 
#         x += 1
#         return "unknown" #We can't know what happens at row 0 as there's no earlier data to inform our decision. 
    
    
    #let's first check what the trade signal is and store it in our local long_short variable. 
    if signal[x] == "no_trade":
        x += 1
        return "no_trade"
    elif signal[x] == "unknown":
        x += 1
        return "no_trade"
    elif signal[x] == "trade" and direction[x] == "long":
        long_short = "long"
    elif signal[x] == "trade" and direction[x] == "short":
        long_short = "short"
    
        
    #Set a counter for the upcoming loop, we want to be able to count out three candles
    short_count = 0 
    long_count = 0 
    
    can_list = [] #create an empty container to store an outcome for the three candles that determine trade success 

    '''
    First we'll focus on the short trade signals. We'll rehash the code use in trade_class
    '''
    
    #Now we find out if a short trade is successful or not based on next 3 candle values
    for i, (j, k, m, n) in enumerate(zip(high[x+1:], low[x+1:], close[x+1:], open_[x+1:])): 
        
        if len(can_list) < 3 and long_short == "short": #check both conditions are true before proceeding. 
            if j > close[x]+drawdown: #We check if the high of the next 30minute candle is higher than our signal candle high

                can_list.append('loss') #add sub_outcome 'loss' to our value_list container. 
                    
            elif close[x] - k > target: #Determines whether we've hit the target

                can_list.append('win')
                
            elif close[x] < m: #Determines whether a reversal signal has been printed
                
                can_list.append('loss')
                
            else:

                can_list.append('no_score') #applicable for a minor retrace or small winning position < 10
        
        elif len(can_list) < 3 and long_short == "long":
            if k < close[x]-drawdown: #We check out if the low of the next 30minute candle is lower than our signal candle

                can_list.append('loss')#if so then add this to the temp_list
                    
            elif j - close[x] > target: #Determine whether we've hit the target

                can_list.append('win')
            
            elif m < close[x]:
                
                can_list.append('loss')
                    
            else:

                can_list.append('no_score')
           
    print(x, can_list)
    '''
    We now look at the can_list elements and check to see what comes first - win or loss. And return whichever of the 
    two outcomes is first. If a win or loss is not recorded we return no score. 
    '''
    if len(can_list)==3: #If our container has stored the next 3 candle outcomes, proceed. 
        if can_list[0] == 'loss':
            x+=1 #increment x by one
            return 'loss' #and return the trade outcome
        elif can_list[0] == 'win':
            x+=1
            return 'win'
        elif can_list[0] == 'no_score' and can_list[1] == 'loss':
            x+=1
            return 'loss'
        elif can_list[0] == 'no_score' and can_list[1] == 'win':
            x+=1
            return 'win'
        elif can_list[0] == 'no_score' and can_list[1] == 'no_score' and can_list[2] == 'loss':
            x+=1
            return 'loss'
        elif can_list[0] == 'no_score' and can_list[1] == 'no_score' and can_list[2] == 'win':
            x+=1
            return 'win'
        else:
            x+=1
            return 'no_score'
    

In [ ]:
#give our arguments some values
profit = 20
stoploss = 20

#pass the trade outcomes to a new dataframe
outcomes = df1.apply(outcome_generator, args=([profit, stoploss]), axis='columns').to_frame()

#join the new dataframe to the end of our central dataframe
df1=df1.join(outcomes)

#give the new feature a descriptive header
df1.rename(columns={0:"outcome"}, inplace=True)

In [ ]:
df1.head()

In [ ]:
df1[df1.outcome=="win"].shape[0]

## Profit Feature

Again we'll only require one new feature to return the max win / loss associated with the parameters set. 

In [ ]:
#Create profit feature using multiple arguments for flexible parameters

df2 = df1

x = 0  #our current row loc stored in a global variable

def profit_calc(row, target, drawdown):
    """
    In this function we will calculate the result of each trade in terms of pips gained or lost. 
    
    """
    global x #let's make our variable global so we can change it's value from inside this function
    
    
    #We'll add a print report to show the progress we're making mid function execute
    if x%250 == 0: 
        print("processing row {}".format(x))
    
    
    #create variables for each of the rows we may need to iterate over
    high = df2['high'] 
    low = df2['low'] 
    close = df2['close']
    open_ = df2['open']
    outcome = df2['outcome']
    direction = df2['direction']
    
   
    
    long_short = "" #set an empty variable to take a long or short position 
    win_loss = "" #set an empty variable to take a win or loss outcome
    
    if x == 0: 
        x += 1
        return 0 #We can't know what happens at row 0 as there's no earlier data to inform our decision. 
    
    
    #let's first check what the trade signal is and store it to our local long_short variable
    if direction[x] == direction[x-1]:
        x += 1
        return 0
    elif direction[x] == "neutral":
        x += 1
        return 0
    elif direction[x] == "short":
        long_short = "short"
    else:
        long_short = "long"
    
    #Check what the outcome was and store this to our local win_loss variable
    if outcome[x] == "no_score":
        x+=1
        return 0
    elif outcome[x] == "win":
        win_loss = "win"
    elif outcome[x] == "loss":
        win_loss = "loss"
    else:
        x+=1
        return 0
        
    #Set counters so we can keep a track of how many candles we've valued    
    short_count = 0 
    long_count = 0 
    
    can_list = [0] #create an empty container to store our trade sub-outcomes
    

   
    #first we'll calculate losses. 
    #A loss is printed when we're stopped out or when a reversal signal is activated. 
    for i, (j, k, m, n, o) in enumerate(zip(high[x+1:], low[x+1:], close[x+1:], open_[x+1:], direction[x+1:])):
        if win_loss == "loss" and long_short == "short":
            #stopped out
            if j > close[x]+drawdown:
                result = - drawdown
                x+=1
                
                return result
            
            #reversal signal so only when the canlde closes in opposite direction to our candle
            elif j < (close[x]+drawdown) and o != direction[x]:
                result = close[x] - m
                x+=1
                
                return result
        
        elif win_loss == "loss" and long_short == "long":
            #stopped out
            if k < close[x]-drawdown:
                result = - drawdown
                x+=1
                
                return result
            
            #reversal signal so only applicable for candles closing in opposite direction to signal
            elif k > (close[x]-drawdown) and o != direction[x]:
                result = m - close[x]
                x+=1
                
                return result
            
    
    #Now we'll calculate wins based on the values of five subsequent candles
    for i, (j, k, m, n, o) in enumerate(zip(high[x+1:], low[x+1:], close[x+1:], open_[x+1:], direction[x+1:])):
        #Check we have less than five entries in our can_list and check that we haven't seen a reversal signal?
        if o == direction[x] and len(can_list) < 6:
            
            
            #our short win is the distance from the signal close to the low of the subsequent 5 candles
            if win_loss == "win" and long_short == "short":
                
                can_list.append(abs(close[x] - k))
            
            #our long win is the distance from the signal close to the high of the subsequent 5 candles
            elif win_loss == "win" and long_short == "long":
                
                can_list.append(abs(j - close[x]))
                
        elif o != direction[x]:
            break
    
    #print the can_list to check our win loop/conditional is performing as expected?
    print(x, can_list, len(can_list))

    
    x+=1    
    
    
    
    #If our can_list has not been populated, we'll just return our target, else we'll return the max value. 
    if len(can_list) < 2:
        return target 
    else:
        return max(can_list)

In [ ]:

df2 = df1
#pass the trade outcomes to a new dataframe
result = df2.apply(profit_calc, args=([profit, stoploss]), axis='columns').to_frame()

#join the new dataframe to the end of our central dataframe
df2=df2.join(result)

#give the new feature a descriptive header
df2.rename(columns={0:"profit"}, inplace=True)

#Review the head of the revised dataframe
#df.head(3)

In [ ]:
df2.head()

In [ ]:
df2.profit.mean()

In [ ]:
# del df2["outcome"]
# del df2["profit"]

In [13]:
df1

,time,open,high,low,close,direction,signal,outcome,profit
0,2020-04-23 11:00:00,5770.0,5822.0,5755.6,5798.1,long,unknown,no_trade,0.0
1,2020-04-23 15:00:00,5797.9,5847.9,5762.1,5773.2,short,trade,loss,-20.0
2,2020-04-23 19:00:00,5773.2,5796.8,5744.1,5769.5,short,no_trade,no_trade,0.0
3,2020-04-23 23:00:00,5756.8,5770.2,5731.6,5740.2,short,no_trade,no_trade,0.0
4,2020-04-24 03:00:00,5740.0,5755.0,5720.3,5744.3,long,trade,win,45.9
...,...,...,...,...,...,...,...,...,...
658,2020-09-27 23:00:00,5892.9,5902.7,5875.5,5901.0,long,no_trade,no_trade,0.0
659,2020-09-28 03:00:00,5901.1,5918.8,5900.2,5916.2,long,no_trade,no_trade,0.0
660,2020-09-28 07:00:00,5916.5,5945.0,5902.8,5921.6,long,no_trade,no_trade,0.0
661,2020-09-28 11:00:00,5921.5,5955.7,5915.1,5944.1,long,no_trade,no_trade,0.0


## Alternate function calls
## CURRENTLY IN USE WITH THE PRIMARY INTERFACE

We have a problem with the kernel dying on the user interface. The solution could be to rewrite the functions so they don't require a global x variable as this may not reset to zero when we rerun the user interface. So we can only run one user interface per kernel which may affect later deployment. 

In [29]:
df.head()

,time,open,high,low,close,direction,signal
0,2020-04-23 11:00:00,5770.0,5822.0,5755.6,5798.1,long,unknown
1,2020-04-23 15:00:00,5797.9,5847.9,5762.1,5773.2,short,trade
2,2020-04-23 19:00:00,5773.2,5796.8,5744.1,5769.5,short,no_trade
3,2020-04-23 23:00:00,5756.8,5770.2,5731.6,5740.2,short,no_trade
4,2020-04-24 03:00:00,5740.0,5755.0,5720.3,5744.3,long,trade


In [326]:
#Create the flexible outcome feature





def outcome_gen(target, drawdown):
   
    
    
    
    #We'll add a print report to show the progress we're making mid function execute
#     if x%200 == 0: 
#         print("processing row {}".format(x))
    
    
    #create variables for each of the rows we may need to iterate over
    high = df["high"]
    low = df["low"]
    close = df["close"]
    open_ = df["open"]
    signal = df["signal"]
    direction = df["direction"]
    
   
    #Set an empty list to hold each of our outcome strings
    outcome = []
    
    
    for x in list(range(0, df.shape[0])):
        
        #set an empty list to store potential winning trades
        can_list = []
    
    #let's first check what the trade signal is and store it in our local long_short variable. 
        if signal[x] == "no_trade":
      
            outcome.append("no_trade")
        
        elif signal[x] == "unknown":
        
            outcome.append("no_trade")
            
        elif x > df.shape[0]-2:
            
            outcome.append("unknown")
            
        else:
            for y in list(range(1,3)):
                
                if signal[x] == "trade" and direction[x] == "long":
                    
                    if low[x+y] < close[x]-drawdown: #We check out if the low of the next 30minute candle is lower than our signal candle

                        can_list.append('loss')#if so then add this to the temp_list
                    
                    elif high[x+y] - close[x] > target: #Determine whether we've hit the target

                        can_list.append('win')
            
                    elif close[x+y] < close[x]:
                
                        can_list.append('loss')
                    
                    else:

                        can_list.append('no_score')
                
                elif signal[x] == "trade" and direction[x] == "short":
                    
                    if high[x+y] > close[x]+drawdown: #We check if the high of the next 30minute candle is higher than our signal candle high

                        can_list.append('loss') #add sub_outcome 'loss' to our value_list container. 
                    
                    elif close[x] - low[x+y] > target: #Determines whether we've hit the target

                        can_list.append('win')
                
                    elif close[x] < close[x+y]: #Determines whether a reversal signal has been printed
                
                        can_list.append('loss')
                
                    else:

                        can_list.append('no_score') #applicable for a minor retrace or small winning position < 10
        
            result = 'no_score'
            for i in can_list:
                if i == "loss":
                    result = i
                    break
                elif i == "win":
                    result = i
                    break
                else:
                    continue
            outcome.append(result)
        
        
        
    
    
           
    #print(x, can_list)
    
    df2 = pd.DataFrame(outcome, columns=["outcome"])
    
    
    return df2

In [328]:
outcome_generator(20, 20).head(10)

,outcome
0,no_trade
1,loss
2,no_trade
3,no_trade
4,win
5,no_trade
6,win
7,win
8,no_trade
9,no_trade


In [329]:
df1 = df.join(outcome_generator(20, 20))

In [330]:
df1.head()

,time,open,high,low,close,direction,signal,outcome
0,2020-04-23 11:00:00,5770.0,5822.0,5755.6,5798.1,long,unknown,no_trade
1,2020-04-23 15:00:00,5797.9,5847.9,5762.1,5773.2,short,trade,loss
2,2020-04-23 19:00:00,5773.2,5796.8,5744.1,5769.5,short,no_trade,no_trade
3,2020-04-23 23:00:00,5756.8,5770.2,5731.6,5740.2,short,no_trade,no_trade
4,2020-04-24 03:00:00,5740.0,5755.0,5720.3,5744.3,long,trade,win


Let's try and build a simple feature without the need for a global x variable

In [331]:
def profit_gen(target, drawdown):
    """
    In this function we'll store all the profits in a list and then join them to the master dataframe
    """
    
    #create a list that holds only the first 0 value for the first row of our profit
    profit = []
    
    #create variables for each of the rows we may need to iterate over
    high = df1["high"]
    low = df1["low"]
    close = df1["close"]
    open_ = df1["open"]
    signal = df1["signal"]
    direction = df1["direction"]
    outcome = df1["outcome"]
    
    
    
    for x in list(range(0, df.shape[0])):
        
        can_list = [target]
        
        if x == 0:
            profit.append(0)
        
        elif x > df.shape[0]-5:
            profit.append(0)
            
        elif signal[x] == "no_trade":
            profit.append(0)
            
        elif direction[x] == "neutral":
            profit.append(0)
            
        elif direction[x] == "short" and outcome[x] == "loss":
            
            #drawdown loss
            if high[x+1] > close[x]+drawdown:
                profit.append(-drawdown)
                
            #reversal loss
            elif high[x+1] < (close[x]+drawdown) and direction[x] != direction[x-1]:
                profit.append(close[x-1] - close[x])
                
            
        elif direction[x] == "long" and outcome[x] == "loss":
            
            #drawdown loss
            if low[x+1] < close[x]-drawdown:
                profit.append(-drawdown)
                
            
            #reversal loss
            elif low[x+1] > (close[x]-drawdown) and direction[x] != direction[x-1]:
                profit.append(close[x+1] - close[x])
        
#         else:
#             profit.append((x, "not loss"))
        
        
        else: 
            for y in list(range(1, 7)):
            
            
            
                if direction[x] == direction[x+y]:
            
                    if outcome[x] == "win" and direction[x] == "short":
                    
                        can_list.append(abs(close[x] - low[x+y]))
                
                    elif outcome[x] == "win" and direction[x] == "long":
                    
                        can_list.append(abs(high[x+y] - close[x]))
            
                elif direction[x] != direction[x+y]:
                
                    break
                
            #print(x, can_list)
        
            profit.append(max(can_list))
    

    
    print(f'the length of profit list is {len(profit)}')
    
    df3 = pd.DataFrame(profit, columns=["profit"])
    
#     df4 = pd.concat([df, df3], ignore_index=True)
    
    return df3
                

In [332]:
simple_profit(20, 20)

the length of profit list is 663


,profit
0,0.0
1,-20.0
2,0.0
3,0.0
4,45.9
...,...
658,0.0
659,0.0
660,0.0
661,0.0


In [333]:
#Now we 
df4 = df1.join(simple_profit(20,20))

the length of profit list is 663


In [334]:
df4

,time,open,high,low,close,direction,signal,outcome,profit
0,2020-04-23 11:00:00,5770.0,5822.0,5755.6,5798.1,long,unknown,no_trade,0.0
1,2020-04-23 15:00:00,5797.9,5847.9,5762.1,5773.2,short,trade,loss,-20.0
2,2020-04-23 19:00:00,5773.2,5796.8,5744.1,5769.5,short,no_trade,no_trade,0.0
3,2020-04-23 23:00:00,5756.8,5770.2,5731.6,5740.2,short,no_trade,no_trade,0.0
4,2020-04-24 03:00:00,5740.0,5755.0,5720.3,5744.3,long,trade,win,45.9
...,...,...,...,...,...,...,...,...,...
658,2020-09-27 23:00:00,5892.9,5902.7,5875.5,5901.0,long,no_trade,no_trade,0.0
659,2020-09-28 03:00:00,5901.1,5918.8,5900.2,5916.2,long,no_trade,no_trade,0.0
660,2020-09-28 07:00:00,5916.5,5945.0,5902.8,5921.6,long,no_trade,no_trade,0.0
661,2020-09-28 11:00:00,5921.5,5955.7,5915.1,5944.1,long,no_trade,no_trade,0.0


## Sandpit

In [106]:
high = df1["high"]
low = df1["low"]
close = df1["close"]
open_ = df1["open"]
signal = df1["signal"]
direction = df1["direction"]
outcome = df1["outcome"]

new_feature = []


for x in list(range(1, 20)):
    can_list = []
    
    for y in list(range(1, 7)):
        
        if high[x+y] > high[x]:
            can_list.append(5)
        
        elif high[x+y] <= high[x]:
            can_list.append(10)
            
        else:
            can_list.append(20)
            
    print(x, can_list)
    new_feature.append(max(can_list))
        
        
        
        

1 [10, 10, 10, 10, 10, 10]
2 [10, 10, 10, 5, 10, 5]
3 [10, 5, 5, 5, 5, 5]
4 [5, 5, 5, 5, 5, 5]
5 [5, 5, 5, 5, 5, 5]
6 [10, 10, 5, 5, 5, 5]
7 [5, 5, 5, 5, 5, 5]
8 [5, 5, 5, 5, 5, 5]
9 [5, 5, 5, 5, 5, 5]
10 [10, 10, 10, 5, 5, 10]
11 [10, 10, 5, 5, 10, 5]
12 [10, 5, 5, 5, 5, 5]
13 [5, 5, 5, 5, 5, 5]
14 [10, 10, 5, 5, 5, 5]
15 [10, 5, 5, 5, 5, 5]
16 [5, 5, 5, 5, 5, 5]
17 [5, 5, 5, 5, 5, 5]
18 [10, 10, 5, 5, 5, 5]
19 [10, 5, 5, 5, 5, 5]


In [105]:
new_feature

[10, 10, 10, 5, 5, 10, 5, 5, 5, 10, 10, 10, 5, 10, 10, 5, 5, 10, 10]

In [128]:
df1.shape[0]

663

In [225]:
import random

In [313]:
choices = ["win", "loss", "no_score"]

r1 = random.choice(choices)
r2 = random.choice(choices)
r3 = random.choice(choices)



In [314]:
can_list = [r1, r1, r1]
can_list

['no_score', 'no_score', 'no_score']

In [315]:
result = 'no_score'
for i in can_list:
    if i == "loss":
        result = i
        break
    elif i == "win":
        result = i
        break
    else:
        continue

result

'no_score'